<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=26fa21e3069e6e44fffed55c4d020d7d9ac754f44255e5caa6e1156bbc42e109
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-05 15:09:27
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: -21.96 K (-0.16%)
Current PnL: -18.42 L (-12.39%)
CY Booked + Current PnL: -8.56 L (-5.76%)
-------------------
Total profit:  2.89 L
Total loss:  -21.31 L
-------------------
Total Booked + Current PnL: 18.42 L (15.14%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.06%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.13 L (56.67%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,8285.00,-12.98,64.0,H-LC,9.67,173800.0,17623.0,8464.0,2.12,11.28,4.87,16.71,22.0,2.08,1.31,30.69,X40N,BTT,HEALTHCARE
25,DIXON,18940.15,-28.58,71.0,H-MC,7.43,142904.0,27968.0,8617.0,0.06,24.33,6.03,31.83,81.0,3.25,1.08,48.12,X40N,ATH,IT
79,TTKPRESTIG,770.00,106.19,67.0,M-SC,6.35,88831.0,-11946.0,12037.0,0.39,-11.85,13.55,0.09,245.0,-0.99,0.67,16.70,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,46.0,M-LC,4.49,213696.0,12216.0,15001.0,-0.37,6.06,7.02,13.51,60.0,0.81,1.61,19.42,XY25,NTT,BANKS
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.61,111968.0,11464.0,19180.0,2.22,11.41,17.13,30.49,79.0,0.60,0.85,47.51,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.68,36.0,H-MC,2.21,134843.0,-945.0,43703.0,-0.53,-0.70,32.41,31.48,107.0,-0.02,1.02,11.34,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.51,60.0,H-MC,1.92,246761.0,-970.0,76027.0,-0.18,-0.39,30.81,30.30,92.0,-0.01,1.86,12.85,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.81,41.0,H-MC,4.14,136352.0,219.0,30911.0,-1.54,0.16,22.67,22.86,91.0,0.01,1.03,12.79,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.02,58.0,H-MC,4.98,202380.0,1560.0,67210.0,-0.24,0.78,33.21,34.25,80.0,0.02,1.53,6.01,X40,ATH,FMCG
88,WIPRO,420.00,-13.79,41.0,M-LC,6.96,151156.0,211.0,109240.0,-0.48,0.14,72.27,72.51,53.0,0.00,1.14,6.17,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-2.37,32.0,H-LC,9.98,222862.0,-19062.0,82325.0,-1.65,-7.88,36.94,26.15,8.0,-0.23,1.68,5.31,X40,ATH,IT
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299892.0,-15950.0,128624.0,-3.97,-5.05,42.89,35.67,5.0,-0.12,2.27,8.10,X40N,ATH,FMCG
45,JIOFIN,387.00,-2.20,39.0,H-LC,13.98,216370.0,-1036.0,54525.0,-0.11,-0.48,25.20,24.61,48.0,-0.02,1.63,54.08,XY24,BTT,FINANCE
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307657.0,-5231.0,176903.0,-1.27,-1.67,57.50,54.87,3.0,-0.03,2.32,4.87,X40,BTT,IT
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286700.0,-59256.0,183115.0,-1.48,-17.13,63.87,35.80,1.0,-0.32,2.17,1.57,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-6.07,51.0,H-LC,1.93,233168.0,-12983.0,92195.0,-0.55,-5.27,39.54,32.18,16.0,-0.14,1.76,28.87,X200,ATH,IT
52,NESTLEIND,1377.00,-8.10,62.0,H-LC,2.55,282649.0,17223.0,39571.0,-0.39,6.49,14.00,21.40,11.0,0.44,2.14,13.56,XY25,NTT,FMCG
43,ITC,452.00,-38.34,47.0,H-LC,3.06,199023.0,-1115.0,22012.0,-2.14,-0.56,11.06,10.44,4.0,-0.05,1.50,4.98,X40,NTT,FMCG
19,CIPLA,1795.00,-20.12,50.0,H-LC,5.53,212925.0,8425.0,32982.0,-1.50,4.12,15.49,20.25,10.0,0.26,1.61,13.21,X40N,BTT,PHARMA
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299892.0,-15950.0,128624.0,-3.97,-5.05,42.89,35.67,5.0,-0.12,2.27,8.10,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.00,1024.40,72.0,M-SC,3.74,152820.0,-28323.0,73797.0,12.38,-15.64,48.29,25.10,235.0,-0.38,1.15,22.47,XY24,NTT,HEALTHCARE
84,VALIANTORG,838.00,-317.09,51.0,H-SC,9.63,92274.0,-41012.0,125603.0,5.00,-30.77,136.12,63.47,139.0,-0.33,0.70,54.67,XR,NTT,CHEMICALS
7,ATULAUTO,844.00,3910.81,78.0,M-SC,9.72,124962.0,-22406.0,74227.0,4.01,-15.20,59.40,35.16,236.0,-0.30,0.94,27.64,XY24,NTT,AUTO
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.61,111968.0,11464.0,19180.0,2.22,11.41,17.13,30.49,79.0,0.60,0.85,47.51,MH,ATH,METALS
4,APOLLOHOSP,8285.00,-12.98,64.0,H-LC,9.67,173800.0,17623.0,8464.0,2.12,11.28,4.87,16.71,22.0,2.08,1.31,30.69,X40N,BTT,HEALTHCARE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299892.0,-15950.0,128624.0,-3.97,-5.05,42.89,35.67,5.0,-0.12,2.27,8.10,X40N,ATH,FMCG
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,36.65,85498.0,-28051.0,68099.0,-3.66,-24.70,79.65,35.27,268.0,-0.41,0.65,109.26,XR,NTT,HOTELS
68,SIS,528.00,2165.80,42.0,H-SC,5.68,60962.0,-20128.0,28798.0,-3.04,-24.82,47.24,10.69,156.0,-0.70,0.46,23.55,OX40N,NTT,MISC
43,ITC,452.00,-38.34,47.0,H-LC,3.06,199023.0,-1115.0,22012.0,-2.14,-0.56,11.06,10.44,4.0,-0.05,1.50,4.98,X40,NTT,FMCG
67,SIEMENS,7969.85,-3.90,46.0,H-LC,30.44,154475.0,-31620.0,244024.0,-2.06,-16.99,157.97,114.13,15.0,-0.13,1.17,13.76,AR,ATH,ELECTRICAL


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,106.19,67.0,M-SC,6.35,88831.0,-11946.0,12037.0,0.39,-11.85,13.55,0.09,245.0,-0.99,0.67,16.70,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,116.52,52.0,M-SC,3.52,142030.0,-32529.0,32567.0,-0.64,-18.63,22.93,0.02,221.0,-1.00,1.07,23.54,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.34,54.0,H-SC,3.48,218736.0,-50931.0,87254.0,-0.08,-18.89,39.89,13.47,138.0,-0.58,1.65,10.63,XY24,NTT,PAINTS
80,UJJIVANSFB,60.0,82.03,41.0,H-SC,13.18,115047.0,-27432.0,46951.0,-1.16,-19.25,40.81,13.70,163.0,-0.58,0.87,35.79,OX40N,NTT,BANKS
18,CERA,9475.0,-20.05,45.0,H-SC,2.78,114030.0,-29773.0,56525.0,-0.47,-20.70,49.57,18.60,149.0,-0.53,0.86,25.84,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.04,41.0,H-LC,14.39,286700.0,-59256.0,183115.0,-1.48,-17.13,63.87,35.80,1.0,-0.32,2.17,1.57,X40,BTT,IT
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307657.0,-5231.0,176903.0,-1.27,-1.67,57.50,54.87,3.0,-0.03,2.32,4.87,X40,BTT,IT
43,ITC,452.00,-38.34,47.0,H-LC,3.06,199023.0,-1115.0,22012.0,-2.14,-0.56,11.06,10.44,4.0,-0.05,1.50,4.98,X40,NTT,FMCG
85,VBL,671.64,-16.73,36.0,H-LC,6.95,299892.0,-15950.0,128624.0,-3.97,-5.05,42.89,35.67,5.0,-0.12,2.27,8.10,X40N,ATH,FMCG
1,ABB,7934.00,-41.33,43.0,H-LC,13.13,245414.0,-16205.0,135419.0,-0.97,-6.19,55.18,45.57,7.0,-0.12,1.85,4.43,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187740.0,36827.0,20482.0,0.24,24.40,10.91,37.97,93.0,1.80,1.42,55.88,AR,ATH,AUTO
25,DIXON,18940.15,-28.58,71.0,H-MC,7.43,142904.0,27968.0,8617.0,0.06,24.33,6.03,31.83,81.0,3.25,1.08,48.12,X40N,ATH,IT
26,DMART,5201.00,-5.89,69.0,H-LC,15.50,235107.0,50626.0,19749.0,-0.65,27.44,8.40,38.14,38.0,2.56,1.78,40.96,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,12.54,63.0,H-MC,3.37,194360.0,29690.0,38269.0,-0.02,18.03,19.69,41.28,89.0,0.78,1.47,27.45,X40N,ATH,AC
86,VOLTAS,1918.49,2.28,64.0,H-MC,3.99,214650.0,22908.0,73131.0,2.02,11.95,34.07,50.08,99.0,0.31,1.62,19.56,XY25,ATH,AC
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187740.0,36827.0,20482.0,0.24,24.40,10.91,37.97,93.0,1.80,1.42,55.88,AR,ATH,AUTO
23,DABUR,735.00,-2.22,63.0,H-MC,4.41,212862.0,15834.0,73778.0,-1.25,8.04,34.66,45.49,102.0,0.21,1.61,22.12,XY24,BTT,FMCG
17,CAMS,5211.76,-5.61,50.0,H-SC,5.47,107391.0,5387.0,38543.0,1.88,5.28,35.89,43.06,122.0,0.14,0.81,23.30,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187740.0,36827.0,20482.0,0.24,24.40,10.91,37.97,93.0,1.80,1.42,55.88,AR,ATH,AUTO
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.61,111968.0,11464.0,19180.0,2.22,11.41,17.13,30.49,79.0,0.60,0.85,47.51,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.65,61.0,M-SC,4.19,101932.0,10434.0,68315.0,0.72,11.40,67.02,86.07,223.0,0.15,0.77,48.02,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.47,47.0,H-SC,12.23,135749.0,12413.0,121346.0,0.00,10.06,89.39,108.45,119.0,0.10,1.03,34.67,AR,ATH,MISC
88,WIPRO,420.00,-13.79,41.0,M-LC,6.96,151156.0,211.0,109240.0,-0.48,0.14,72.27,72.51,53.0,0.00,1.14,6.17,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,36.65,85498.0,-28051.0,68099.0,-3.66,-24.70,79.65,35.27,268.0,-0.41,0.65,109.26,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.34,45.0,H-SC,15.65,89113.0,-11850.0,106383.0,-1.28,-11.74,119.38,93.63,148.0,-0.11,0.67,32.73,SR,ATH,CHEMICALS
50,MASFIN,398.61,-18.25,51.0,H-SC,12.87,93015.0,-4965.0,26565.0,-0.19,-5.07,28.56,22.05,152.0,-0.19,0.70,35.20,XR,ATH,FINANCE
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.61,111968.0,11464.0,19180.0,2.22,11.41,17.13,30.49,79.0,0.60,0.85,47.51,MH,ATH,METALS
80,UJJIVANSFB,60.00,82.03,41.0,H-SC,13.18,115047.0,-27432.0,46951.0,-1.16,-19.25,40.81,13.70,163.0,-0.58,0.87,35.79,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,57.0,L-SC,36.65,85498.0,-28051.0,68099.0,-3.66,-24.70,79.65,35.27,268.0,-0.41,0.65,109.26,XR,NTT,HOTELS
7,ATULAUTO,844.00,3910.81,78.0,M-SC,9.72,124962.0,-22406.0,74227.0,4.01,-15.20,59.40,35.16,236.0,-0.30,0.94,27.64,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,21.09,187740.0,36827.0,20482.0,0.24,24.40,10.91,37.97,93.0,1.80,1.42,55.88,AR,ATH,AUTO
57,RELAXO,1176.00,-31.46,62.0,H-SC,4.97,63616.0,-51922.0,86912.0,-0.26,-44.94,136.62,30.28,136.0,-0.60,0.48,25.07,X40N,NTT,FOOTWEAR
51,NATIONALUM,247.44,-43.66,74.0,H-MC,0.61,111968.0,11464.0,19180.0,2.22,11.41,17.13,30.49,79.0,0.60,0.85,47.51,MH,ATH,METALS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,43.70
2,50,73.93


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.99
LC    36.11
MC    25.90
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.87
X40      15.61
X40N     14.31
XY25     10.14
AR        9.49
XR        9.06
OX40N     7.44
MH        1.78
X200      1.76
X5K       1.44
SR        1.10
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.84
H-MC    22.88
H-SC    22.61
M-SC    13.82
M-LC     7.17
M-MC     2.69
L-SC     1.56
L-LC     1.10
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.90,-0.36,34.80
IT,13.40,-16.32,73.99
FINANCE,7.42,-22.78,68.04
BANKS,7.37,-16.86,66.48
PAINTS,6.09,-14.51,40.14
MISC,5.87,-18.53,86.47
ELECTRICAL,5.68,-9.33,69.59
AUTO,4.83,-5.53,51.44
AC,3.83,4.37,28.02


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2775774.0,22
AR,1222195.0,10
XR,1095983.0,13
X40,899987.0,10
X40N,539770.0,11
OX40N,473250.0,10
XY25,409396.0,7
SR,197314.0,2
X5K,126491.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2783048.0,27
H-LC,1435336.0,16
M-SC,1415713.0,17
H-MC,1177028.0,17
M-LC,427240.0,5
M-MC,321685.0,2
L-SC,254852.0,3
L-MC,59506.0,1
L-LC,38897.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,934956.0,6
M-SC,XY24,830419.0,7
H-SC,AR,661005.0,5
H-LC,X40,611307.0,5
H-SC,XR,525380.0,6
H-MC,XY24,457681.0,4
H-LC,AR,379443.0,2
M-MC,XY24,321685.0,2
H-LC,X40N,233610.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
